# Run AlphaFold 3 (Singularity) for One Complex

This notebook runs a single AlphaFold 3 prediction using the same bind-mount layout as the working HPC script:

- Host input dir -> `/root/af_input`
- Host output dir -> `/root/af_output`
- Model parameters -> `/root/models`
- Databases -> `/root/public_databases`
- AF3 program dir -> `/root/AF3`

Target input JSON:
- `af3_inputs/P1A8_1JPLm414_C_G1.json`

Notes:
- `--run_data_pipeline` is CPU-only (genetic/template search).
- `--run_inference` requires a GPU.

In [ ]:
import json
import os
import subprocess
import sys
from pathlib import Path
from datetime import datetime

# -----------------
# User config
# -----------------
input_json = Path("af3_inputs/P1A8_1JPLm414_C_G1.json")
run_name = input_json.stem

# Where to store AF3 outputs on the host
host_output_dir = Path("af3_outputs") / run_name

# AF3 installation paths (based on the working script you shared)
AF3_HOME = Path("/opt/apps/community/alphafold3")
DATABASES_DIR = AF3_HOME / "AF3_databases"
MODEL_PARAMETERS_DIR = AF3_HOME / "AF3_models"
AF3_PROGRAMS_DIR = AF3_HOME / "AF3_Programs" / "alphafold3"

# Pick the SIF image that exists on this system
# Observed on dhvi: AF3_v3.0.1.sif and AF3_111125.sif
for candidate in [
    AF3_HOME / "AF3_v3.0.1.sif",
    AF3_HOME / "AF3_111125.sif",
    AF3_HOME / "AF_v3.0.1.sif",
]:
    if candidate.exists():
        SIF_IMAGE = candidate
        break
else:
    SIF_IMAGE = AF3_HOME / "AF3_v3.0.1.sif"  # fallback; will fail validation if missing

# Execution control flags
run_data_pipeline = True
run_inference = True

# Set to True to only print the command (no execution)
dry_run = False

# Optional Apptainer cache dirs (safe to leave as-is)
user = os.environ.get("USER", "unknown")
APPTAINER_TMPDIR = Path(f"/scratch/{user}/tmp")
APPTAINER_CACHEDIR = Path(f"/cwork/{user}/cache")

# AF3 uses Python's tempfile which defaults to /tmp or TMPDIR
# Set TMPDIR to a clean location to avoid cleanup issues
AF3_TMPDIR = Path(f"/scratch/{user}/af3_tmp")

print("Config")
print("=" * 80)
print(f"Input JSON: {input_json}")
print(f"Run name: {run_name}")
print(f"Host output dir: {host_output_dir}")
print(f"AF3_HOME: {AF3_HOME}")
print(f"SIF_IMAGE: {SIF_IMAGE}")
print(f"DATABASES_DIR: {DATABASES_DIR}")
print(f"MODEL_PARAMETERS_DIR: {MODEL_PARAMETERS_DIR}")
print(f"AF3_PROGRAMS_DIR: {AF3_PROGRAMS_DIR}")
print(f"run_data_pipeline: {run_data_pipeline}")
print(f"run_inference: {run_inference}")
print(f"dry_run: {dry_run}")

Config
Input JSON: af3_inputs/P1A8_1JPLm414_C_G1.json
Run name: P1A8_1JPLm414_C_G1
Host output dir: af3_outputs/P1A8_1JPLm414_C_G1
AF3_HOME: /opt/apps/community/alphafold3
SIF_IMAGE: /opt/apps/community/alphafold3/AF3_v3.0.1.sif
DATABASES_DIR: /opt/apps/community/alphafold3/AF3_databases
MODEL_PARAMETERS_DIR: /opt/apps/community/alphafold3/AF3_models
AF3_PROGRAMS_DIR: /opt/apps/community/alphafold3/AF3_Programs/alphafold3
run_data_pipeline: True
run_inference: True
dry_run: False


In [ ]:
# -----------------
# Validation
# -----------------

if not input_json.exists():
    raise FileNotFoundError(f"Missing input JSON: {input_json}")

for label, p in [
    ("SIF_IMAGE", SIF_IMAGE),
    ("DATABASES_DIR", DATABASES_DIR),
    ("MODEL_PARAMETERS_DIR", MODEL_PARAMETERS_DIR),
    ("AF3_PROGRAMS_DIR", AF3_PROGRAMS_DIR),
]:
    if not p.exists():
        raise FileNotFoundError(f"Missing {label}: {p}")

host_output_dir.mkdir(parents=True, exist_ok=True)

# Optional cache dirs
try:
    APPTAINER_TMPDIR.mkdir(parents=True, exist_ok=True)
    os.environ["APPTAINER_TMPDIR"] = str(APPTAINER_TMPDIR)
except Exception:
    pass

try:
    APPTAINER_CACHEDIR.mkdir(parents=True, exist_ok=True)
    os.environ["APPTAINER_CACHEDIR"] = str(APPTAINER_CACHEDIR)
except Exception:
    pass

# Set TMPDIR for AF3's Python tempfile operations
# This helps avoid cleanup errors with temp directories
try:
    AF3_TMPDIR.mkdir(parents=True, exist_ok=True)
    os.environ["TMPDIR"] = str(AF3_TMPDIR)
    print(f"Set TMPDIR to: {AF3_TMPDIR}")
except Exception as e:
    print(f"Warning: Could not set TMPDIR: {e}")

# Validate JSON contents
with open(input_json, "r") as f:
    data = json.load(f)

seq_ids = [s.get("protein", {}).get("id") for s in data.get("sequences", [])]
required = {"H", "L", "Ag"}
if set(seq_ids) != required:
    raise ValueError(f"Expected chains {sorted(required)}, got {seq_ids}")

print("OK: inputs validated")
print(f"  JSON name: {data.get('name')}")
print(f"  JSON version: {data.get('version')}")
print(f"  chains: {seq_ids}")

OK: inputs validated
  JSON name: P1A8_1JPLm414_C_G1
  JSON version: 3
  chains: ['H', 'L', 'Ag']


In [ ]:
# -----------------
# Build command (adapted from working HPC script)
# -----------------

# Bind host input dir (containing the JSON) to /root/af_input
host_input_dir = input_json.parent
json_in_container = f"/root/af_input/{input_json.name}"

flags = [
    f"--json_path={json_in_container}",
    "--model_dir=/root/models",
    "--db_dir=/root/public_databases",
    "--output_dir=/root/af_output",
]

if not run_data_pipeline:
    flags.append("--run_data_pipeline=false")
if not run_inference:
    flags.append("--run_inference=false")

# Bind temp directory if we set one
bind_args = [
    "--bind", f"{host_input_dir.absolute()}:/root/af_input",
    "--bind", f"{host_output_dir.absolute()}:/root/af_output",
    "--bind", f"{MODEL_PARAMETERS_DIR}:/root/models",
    "--bind", f"{DATABASES_DIR}:/root/public_databases",
    "--bind", f"{AF3_PROGRAMS_DIR}:/root/AF3",
]

# If TMPDIR is set, bind it so AF3 can use it inside container
# Also bind it to /tmp (the default temp location) so Python tempfile uses it
if "TMPDIR" in os.environ:
    tmpdir_host = Path(os.environ["TMPDIR"])
    # Bind to both /tmp/af3_tmp and /tmp so AF3's Python tempfile uses it
    bind_args.extend([
        "--bind", f"{tmpdir_host.absolute()}:/tmp/af3_tmp",
        "--bind", f"{tmpdir_host.absolute()}:/tmp",
    ])

cmd_parts = [
    "singularity", "exec", "--nv",
] + bind_args + [
    str(SIF_IMAGE),
    "python", "/root/AF3/run_alphafold.py",
] + flags

print("Command")
print("=" * 80)
print(" ".join(cmd_parts))
print("=" * 80)

Command
singularity exec --nv --bind /cwork/hsb26/ab_seq_bind_analysis/af3_inputs:/root/af_input --bind /cwork/hsb26/ab_seq_bind_analysis/af3_outputs/P1A8_1JPLm414_C_G1:/root/af_output --bind /opt/apps/community/alphafold3/AF3_models:/root/models --bind /opt/apps/community/alphafold3/AF3_databases:/root/public_databases --bind /opt/apps/community/alphafold3/AF3_Programs/alphafold3:/root/AF3 /opt/apps/community/alphafold3/AF3_v3.0.1.sif python /root/AF3/run_alphafold.py --json_path=/root/af_input/P1A8_1JPLm414_C_G1.json --model_dir=/root/models --db_dir=/root/public_databases --output_dir=/root/af_output


In [ ]:
# -----------------
# Run AF3
# -----------------

run_info = {
    "timestamp_start": datetime.now().isoformat(),
    "input_json": str(input_json.absolute()),
    "host_input_dir": str(host_input_dir.absolute()),
    "host_output_dir": str(host_output_dir.absolute()),
    "cmd": " ".join(cmd_parts),
    "run_data_pipeline": run_data_pipeline,
    "run_inference": run_inference,
}

if dry_run:
    print("Dry run enabled; not executing.")
else:
    print(f"Starting: {run_info['timestamp_start']}")
    
    # Pass environment variables (including TMPDIR) to subprocess
    # This ensures AF3 inside the container uses our temp directory
    env = os.environ.copy()
    if "TMPDIR" in env:
        # Also set TMPDIR explicitly in the environment passed to Singularity
        # Singularity will pass this to the container
        print(f"Passing TMPDIR={env['TMPDIR']} to container")
    
    proc = subprocess.Popen(
        cmd_parts,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        env=env,
    )

    output_lines = []
    for line in proc.stdout:
        print(line, end="")
        output_lines.append(line)
        sys.stdout.flush()

    rc = proc.wait()
    run_info["timestamp_end"] = datetime.now().isoformat()
    run_info["return_code"] = rc
    run_info["output_tail"] = output_lines[-200:]

    print("\n" + "=" * 80)
    if rc == 0:
        print("SUCCESS: AF3 completed")
    else:
        print(f"FAILURE: AF3 exited with code {rc}")
    print(f"Finished: {run_info['timestamp_end']}")

Starting: 2026-01-28T14:55:15.116197
I0128 14:55:21.515559 139737590023296 xla_bridge.py:895] Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
I0128 14:55:21.544439 139737590023296 xla_bridge.py:895] Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory

Running AlphaFold 3. Please note that standard AlphaFold 3 model parameters are
only available under terms of use provided at
https://github.com/google-deepmind/alphafold3/blob/main/WEIGHTS_TERMS_OF_USE.md.
If you do not agree to these terms and are using AlphaFold 3 derived model
parameters, cancel execution of AlphaFold 3 inference with CTRL-C, and do not
use the model parameters.

Found local devices: [CudaDevice(id=0)], using device 0: cuda:0
Building model from scratch...
Checking that model parameters can be loaded...

Running fold job P1A8_1JPLm414_C_G1...
Output will be written in /root/

In [ ]:
# -----------------
# Inspect outputs + save run info
# -----------------

files = sorted(host_output_dir.glob("**/*"))
print(f"Output dir: {host_output_dir}")
print(f"Files found: {len(files)}")

for f in files[:50]:
    rel = f.relative_to(host_output_dir)
    if f.is_file():
        print(f"  - {rel} ({f.stat().st_size} bytes)")
    else:
        print(f"  - {rel}/")

run_info_path = host_output_dir / "run_info.json"
with open(run_info_path, "w") as f:
    json.dump(run_info, f, indent=2)

print(f"\nSaved run info: {run_info_path}")

## Notes on Temp Directory Cleanup Errors

If you see `OSError: [Errno 39] Directory not empty` errors:

1. **This is often non-fatal** - AF3 may have already written outputs before the cleanup error
2. **Check outputs anyway** - Look in the output directory for PDB files or other results
3. **Clean temp dirs** - You can manually clean `/hpc/home/hsb26/tmp/tmp*` directories if they accumulate
4. **The notebook now sets TMPDIR** to `/scratch/{user}/af3_tmp` to isolate AF3 temp files

The error occurs during cleanup of temporary directories created during template search. The actual structure prediction may still complete successfully.

## Notes

If you see errors about missing models/databases, verify that these exist:
- `/opt/apps/community/alphafold3/AF3_models`
- `/opt/apps/community/alphafold3/AF3_databases`

If you want to run only CPU search on a CPU node:
- set `run_inference = False`

If you want to run only GPU inference (after data pipeline is complete):
- set `run_data_pipeline = False`